In [1]:
#file navigation tools
from glob import glob
import os
#image analysis library
from skimage import io
#jupyter notebook img display
import stackview
#The fundamental package for scientific computing with Python
import numpy as np
#python image viewer 
import napari
#excel for python
import pandas as pd
import csv

import tifffile

import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot

import cv2 as cv
from typing import List


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg

In [2]:
def lap_variance(img):
    return np.var(cv.Laplacian(img, cv.CV_64F, ksize=21))

In [3]:
def best_focus_z_slice_id(channel):
    lap_var_z_slice = []
    for img in channel:
        lap_var_z_slice.append(lap_variance(img)) 
    max_var = max(lap_var_z_slice)
    max_var_id = lap_var_z_slice.index(max_var)
    return max_var_id

In [4]:
def read_stack(folder):
    files=glob(folder)
    img = io.imread(files[0], plugin='pil')
    img=np.expand_dims(img,1)
    img=np.reshape(img,(4,41,2304,2304))
    return files, img

In [5]:
def choose_focus_lap(channel_stack):
    focus = [best_focus_z_slice_id(channel_stack)-10, best_focus_z_slice_id(channel_stack)+10]
    #BIGFISH
    #focus = stack.compute_focus(img[i], neighborhood_size=31)
    return focus

In [6]:
def np_mip(channel_array, focus):
    return np.amax(channel_array[focus[0]:focus[1],...],axis=0,keepdims=False)

In [7]:
def projection_filter(channel_projection):
    return stack.remove_background_gaussian(channel_projection, sigma=3)

In [8]:
def napari_view(files, zproject, channels=['CY5', 'CY3', 'CY3.5', 'DAPI'], colors=['magenta', 'green', 'cyan', 'blue']):
    viewer= napari.Viewer()
    viewer.add_image(io.imread(files[1]), name='DIC', opacity=0.30)
    for i in range(zproject.shape[0]):
        viewer.add_image(zproject[i,...],name=channels[i],colormap=colors[i],opacity=0.90)
        

In [9]:
def split_stack(image, channels=['CY5', 'CY3', 'CY3.5', 'DAPI'], colors=['magenta', 'green', 'cyan', 'blue']):
    image_dictionary = {}
    print(image.shape)
    for i in range(0,len(image)):
        print(i)
        image_dictionary[channels[i]] = (colors[i], image[i])
    return image_dictionary    
    

In [44]:
corresponding_files, stack_example = read_stack("C:/Users/lotta/Documents/Bioinformatics_Msc/Project/DATA_INITIAL/pair_2/*")
channel_dictionary = split_stack(stack_example)
#channel_dictionary['DAPI'][1]
projection = (np_mip(channel_dictionary['DAPI'][1], choose_focus_lap(channel_dictionary['DAPI'][1])))

(4, 41, 2304, 2304)
0
1
2
3


In [41]:
napari_view(corresponding_files, np.array([projection]), channels=['DAPI'], colors=['blue'])

In [ ]:


# assuming your layer.data is a 3D array, with the first axis as Z
for i, plane in enumerate(layer.data):
    tifffile.imsave(f'my_image_z{i:03}.tif', plane)

In [46]:
import os
os.chmod("C:/Users/lotta/Bioinformatics", 755)
cellpose_input = [io.imread(corresponding_files[1]), projection]
io.imsave("C:/Users/lotta/Pictures/Cellpose/pair2_pic.tiff", np.array(cellpose_input), 'tifffile')

In [45]:
alpha = 0.95
image0 = io.imread(corresponding_files[1])
image1 = projection
blended = alpha * image0 + (1 - alpha) * image1
cellpose_input = [blended, image1]
io.imsave("C:/Users/lotta/Pictures/Cellpose/blended2channel_pair2_pic.tiff", np.array(cellpose_input), 'tifffile')

In [47]:
corresponding_files, stack_example = read_stack("Z:/bigdata/2022/Marah/220601/yET563 BR1/TIFF/*")